  ## *RAG WITH MULTI DATA SOURCE*

In [1]:
# #to access the tools like wikipedia arxiv we will be creating wrapper for individual tools in 
# top of it and later we will combine it with user and llm to create chain

In [2]:
from langchain_community.tools import WikipediaQueryRun
#this will be responsible for running the wikipediaapi wrapper

In [3]:
from langchain_community.utilities import WikipediaAPIWrapper
# this wrapper will hit wikipedia api and will provide the response according to my query

In [4]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
wiki.name

'wikipedia'

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter



/home/santosh/ml_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-22 21:05:57.157482: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-22 21:05:57.663768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-22 21:06:00.520351: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different nu

In [7]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/tmp/ipykernel_8642/1474760240.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [8]:
loader=WebBaseLoader("https://pmc.ncbi.nlm.nih.gov/articles/PMC7983091/")
docs=loader.load()
##create chunks of docs 
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()##this one is default retriever tool it's a faiss retriever not a langchian tool 
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7264c039f8c0>, search_kwargs={})

In [56]:

# ##we will create our own retriever tool 

from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pmc_retriever",
    description="Search for info about machine learning algorithms and their real world applications and research directions."
)


In [57]:
retriever_tool.name

'pmc_retriever'

In [58]:
##arxiv tool 
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [59]:
tools=[retriever_tool,arxiv,wiki]#[retriever_tool,wiki,arxiv] only retriever tool for now 

In [60]:
tools

[StructuredTool(name='pmc_retriever', description='Search for info about machine learning algorithms and their real world applications and research directions.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x72639911a8e0>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x72639911a5c0>),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/santosh/ml_env/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))]

In [61]:
##import environment variables
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

In [62]:
# #we can pull prompt template from hub also 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful AI assistant. Use tools when necessary."),
#     MessagesPlaceholder(variable_name="chat_history"),
#     ("human", "{input}"),
#     MessagesPlaceholder(variable_name="agent_scratchpad"),
# ])


In [63]:
##AGENTS
from langchain.agents import create_agent
agent=create_agent(
    model,
    tools,
    system_prompt = """
You are a helpful AI assistant. 

Rules:
 
1. For reasoning, document-based, or multi-step questions, use tools (retriever, wiki, arxiv) as needed.  
2. Be concise, accurate, and structured in your answers. 
3. Never call more than one tool per question unless absolutely necessary. 
4. Always provide references from documents when requested.
"""

)

In [66]:
response = agent.invoke({
    'messages': [{
        'role': 'user',
        'content': "What are the recent advances in representation learning using self-supervised methods, and how do they compare with supervised approaches?"
    }]
})


In [67]:
response['messages'][-1].content

"Self-supervised learning (SSL) is a paradigm in machine learning where a model is trained on a task using the data itself to generate supervisory signals, rather than relying on human-provided labels. This approach has gained popularity in recent years due to its ability to learn representations that are useful for a wide range of tasks, without the need for large amounts of labeled data.\n\nOne of the key advantages of SSL is that it can be used to learn representations that are invariant to certain transformations, such as rotations or translations. This is because the model is trained to predict the output of a transformation, rather than the transformation itself. For example, a model trained on images of objects with different rotations would learn to represent the objects in a way that is invariant to rotation.\n\nSSL can be used in a variety of applications, including computer vision, natural language processing, and speech recognition. It has been shown to be particularly effe

In [68]:
response

{'messages': [HumanMessage(content='What are the recent advances in representation learning using self-supervised methods, and how do they compare with supervised approaches?', additional_kwargs={}, response_metadata={}, id='864f64cb-58dc-44a7-8cda-120a5590828e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '8nm150kmt', 'function': {'arguments': '{"query":"Recent advances in representation learning using self-supervised methods and comparison with supervised approaches."}', 'name': 'pmc_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 616, 'total_tokens': 648, 'completion_time': 0.055711948, 'completion_tokens_details': None, 'prompt_time': 0.034180196, 'prompt_tokens_details': None, 'queue_time': 0.049012634, 'total_time': 0.089892144}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provi